<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/doubt_marker_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
random.seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)



In [2]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:

%cd /content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/3_Classification

%ls

/content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/3_Classification
annotation_Drew_sample_doubt_markers.csv  gold_standard_doubt_marker_100.csv
doubt_marker_accuracy.gsheet              gold_standard_doubt_marker_500.csv
doubt_marker_classification.ipynb         predicted_mega.csv
drew_gold_standard_doubt_markers_500.csv  predictions.csv


In [4]:
# Read the Annie Sample csv file
df1 = pd.read_csv("gold_standard_doubt_marker_500.csv") #Annie Sample

# Read the Drew Sample csv file
df2 = pd.read_csv("drew_gold_standard_doubt_markers_500.csv") #Drew Sample
# Bind the rows of the two dataframes
df = pd.concat([df1, df2], ignore_index=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sentence_id   1003 non-null   object 
 1   sentence      997 non-null    object 
 2   matched_term  997 non-null    object 
 3   doubt_label   997 non-null    float64
dtypes: float64(1), object(3)
memory usage: 31.5+ KB


In [5]:

df['label'] = df['doubt_label']
df.drop(['sentence_id','doubt_label'], axis=1)
del df['sentence_id'], df['doubt_label']
# Drop rows with any NaN values
df.dropna(inplace=True)
df.head(3)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 1002
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sentence      997 non-null    object 
 1   matched_term  997 non-null    object 
 2   label         997 non-null    float64
dtypes: float64(1), object(2)
memory usage: 31.2+ KB


In [6]:
cols = ['sentence', 'matched_term']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df.head(3)

,sentence,matched_term,label,text
0,"Mom is ambivalent about pumping, and does not ...",\bambivalent\b,1.0,"Mom is ambivalent about pumping, and does not ..."
1,PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...,\badamantly\b,0.0,PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...
2,believes th Amiodarone extreme tremors Precaut...,\bbelieves\b,1.0,believes th Amiodarone extreme tremors Precaut...


In [7]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [8]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #print(self.targets.shape)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [9]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 1002
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    997 non-null    object 
 1   label   997 non-null    float64
dtypes: float64(1), object(1)
memory usage: 23.4+ KB


In [10]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (997, 2)
TRAIN Dataset: (798, 2)
TEST Dataset: (199, 2)


In [11]:
train_data['label'].unique()

array([0., 1.])

In [12]:
test_data['label'].unique()

array([1., 0.])

In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2) #output labels = 1 for binary classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [15]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [16]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [18]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [19]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  1.53it/s]

Training Loss: 0.6422290205955505
Training Accuracy: 100.0


2it [00:00,  2.54it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
7it [00:05,  2.04it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max

The Total Accuracy for Epoch 0: 71.17794486215539
Training Loss Epoch: 0.5283820936083794
Training Accuracy Epoch: 71.17794486215539


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

The Total Accuracy for Epoch 3: 86.96741854636592
Training Loss Epoch: 0.350050110751763
Training Accuracy Epoch: 86.96741854636592


1it [00:00,  7.90it/s]

Training Loss: 0.5803136229515076
Training Accuracy: 75.0


4it [00:00,  5.22it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
9it [00:01,  4.89it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max

The Total Accuracy for Epoch 4: 89.22305764411027
Training Loss Epoch: 0.30061933370307087
Training Accuracy Epoch: 89.22305764411027


In [20]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [21]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu


Initial Run -- one interation

In [22]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2it [00:00, 18.33it/s]

Validation Loss: 0.29413896799087524
Validation Accuracy: 75.0


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
24it [00:01, 16.69it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pa

Validation Loss Epoch: 0.38303235188126566
Validation Accuracy Epoch: 82.91457286432161
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       131
           1       0.75      0.75      0.75        68

    accuracy                           0.83       199
   macro avg       0.81      0.81      0.81       199
weighted avg       0.83      0.83      0.83       199

Confusion Matrix:
[[114  17]
 [ 17  51]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(48156), tensor(14686), tens...         1          1
1  [tensor(0), tensor(20158), tensor(2046), tenso...         0          0
2  [tensor(0), tensor(12295), tensor(13875), tens...         1          1
3  [tensor(0), tensor(20823), tensor(592), tensor...         0          1
4  [tensor(0), tensor(10350), tensor(1350), tenso...         1          0


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2it [00:00, 18.61it/s]

Validation Loss: 0.07705655694007874
Validation Accuracy: 100.0


8it [00:00, 17.41it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
10it [00:01,  5.20it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave ma

Validation Loss Epoch: 0.383040135204792
Validation Accuracy Epoch: 82.91457286432161
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       131
           1       0.75      0.75      0.75        68

    accuracy                           0.83       199
   macro avg       0.81      0.81      0.81       199
weighted avg       0.83      0.83      0.83       199

Confusion Matrix:
[[114  17]
 [ 17  51]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(12550), tensor(108), tensor...         0          0
1  [tensor(0), tensor(18276), tensor(4843), tenso...         0          0
2  [tensor(0), tensor(16261), tensor(448), tensor...         0          0
3  [tensor(0), tensor(487), tensor(4668), tensor(...         1          1
4  [tensor(0), tensor(22985), tensor(3215), tenso...         0          1
test accuracy = 82.91%


In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [23]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]], dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)

        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df


In [24]:
predicted_classes = predict_on_dataframe(newdf, model, tokenizer, max_len=128)
print(predicted_classes)

annotation_drew_sample = pd.read_csv("annotation_Drew_sample_doubt_markers.csv")
annotation_drew_sample.info()
annotation_drew_sample= annotation_drew_sample.rename(columns={'Sentence':'text'})

predicted_mega = predict_on_dataframe(annotation_drew_sample, model, tokenizer, max_len=128)
predicted_mega
#
filtered_df = predicted_mega[predicted_mega['predictions'] == 1]

filtered_df

predicted_mega.to_csv(r'predicted_mega.csv', index = False)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

                                                   text  label  predictions
0     Mom is ambivalent about pumping, and does not ...    1.0            1
1     PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...    0.0            0
2     believes th Amiodarone extreme tremors Precaut...    1.0            0
3     [**Doctor First Name 955**] states she believe...    0.0            0
4     Neurosurgery believes that this is the source ...    0.0            0
...                                                 ...    ...          ...
998   believes she is her to detox off of methadone ...    0.0            0
999         PT INSISTING TO BE SUPINE.</s>\binsisting\b    1.0            1
1000  After sheath d/c became acutely agitated, clai...    1.0            1
1001   She believes she last moved her bowels this m...    0.0            0
1002  INR 1.2 Plan: Most likely CT today to follow b...    0.0            0

[997 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [25]:

# constant for classes
#classes = ('Neither Source/Location', 'Source, no user', 'User, no source', 'User and source')


#print(test_data["label"])
#predicted_roberta = model.
#predict(test_data["text"])
#print(predicted)
# Build confusion matrix
#cf_matrix = confusion_matrix(test_data["label"], predicted)
#df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
#                     columns = [i for i in classes])
#plt.figure(figsize = (4,4))
#sn.heatmap(df_cm, annot=True)
#plt.savefig('output.png')

Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [26]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [27]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [28]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
